# Chazam

<img src='images/chazam.png' width='500'/>

### 01 - Audio

### 02 - Señal (waveform)

### 03 - Espectrograma

![eq](images/eq.gif)

### 04 - Extraer picos en el espectrograma

### 05 - Hashing fingerprints

<img src='images/fingerprint.jpg' width='500'/>

[funcion hash](https://es.wikipedia.org/wiki/Funci%C3%B3n_hash)


$hashing(\text{picos}, \text{diferencia tiempo entre picos}) = \text{valor hash del fingerprint, tiempo inicial}$

### 06 - Base de datos (SQL)

**Creación tabla de canciones**


```sql
create table if not exists songs (
                
                song_id mediumint unsigned not null auto_increment,
                song_name varchar(250) not null,
                fingerprinted tinyint default 0,
                file_sha1 binary(20) not null,
                total_hashes int not null default 0,
                date_created  datetime not null default current_timestamp,
                date_modified datetime not null default current_timestamp on update current_timestamp,

               constraint pk_songs_song_id primary key (song_id),
               constraint uq_songs_song_id unique key (song_id)
    
         );
```

**Creacion tabla de fingerprints**

```sql
create table if not exists fingerprints (
    
                    hash binary(10) not null,
                    song_id mediumint unsigned not null,
                    offset int unsigned not null,
                    date_created  datetime not null default current_timestamp,
                    date_modified datetime not null default current_timestamp on update current_timestamp,
                   
    
   index ix_fingerprints_hash (hash),

   constraint uq_fingerprints_song_id_offset_hash unique key (song_id, offset, hash),
   constraint fk_fingerprints_song_id foreign key (song_id) references fingerprints(song_id) on delete cascade
            );

```

**Inserción de datos en tabla canciones**

```sql
insert into songs (song_name, file_sha1, total_hashes) 
       values (song_name_data, UNHEX(file_sha1_data), total_hashes_data);
```

**Inserción de datos en tabla fingerprints**

```sql
insert ignore into fingerprints (hash, song_id, offset) 
              values (UNHEX(hash_data), song_id_data, offset_data);
```

**búsqueda de canción**

```sql
select HEX(hash), song_id, offset from fingerprints where hash in (hash_data);
```

### 07 - Detección

### 08 - API (deploy)

```python

from flask import Flask, request
from flask_cors import CORS        # cross-origin resource sharing

from chazam import Chazam

import json

from config import WAV_PATH, COVERS, DATA_DEFAULT


app=Flask(__name__)

cors=CORS(app, resources={r'*': {'origins': '*'}})

ih_chazam=Chazam()


@app.route('/', methods=['POST', 'GET'])
def main():

    audio=request.files['blob']
    audio.save(WAV_PATH)

    song=ih_chazam.recognize(WAV_PATH)

    if song['results'][0]['input_confidence']<0.04:
        return json.dumps(DATA_DEFAULT), 200

    else:

        song_name=song['results'][0]['song_name'].decode('utf-8').split('-')

        data={'artist': song_name[0].strip(),
              'song': song_name[1].strip(),
              'cover': COVERS[song['results'][0]['song_name'].decode('utf-8')],
              'match_time': song['total_time']
              }

        return json.dumps(data), 200

   
if __name__== '__main__':
    app.run(debug=True)




```